In [1]:
import numpy as np
from numba import jit, vectorize, float64, int64
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install -i https://test.pypi.org/simple/ presnie

Looking in indexes: https://test.pypi.org/simple/


In [4]:
from algorithm import sghmc

In [5]:
def generate_data(nrow,ncol):
    np.random.seed(1234)
    X=np.random.normal(0,1,[nrow,ncol])  
    Y=np.random.binomial(1,1/(1+np.exp(-(X@theta))))
    test=np.random.choice(range(nrow),int(nrow/2),replace=False)
    train=np.array(list(set(range(nrow))-set(test)))
    return X, Y, test,train

In [6]:
np.random.seed(1234)
theta=np.array([-5,10,5,20,30])   
X,Y,test,train = generate_data(20000,5)

In [7]:
X_train=X[train,:]
Y_train=Y[train]
X_test=X[test,:]
Y_test=Y[test]

## Optimization

In [8]:
@jit
def batch_numba(X,Y, nbatch):
    nrow = X.shape[0]
    idx = np.random.choice(nrow, nbatch, replace = False)
    X_sample = X[idx,:]
    Y_sample = Y[idx]
    return X_sample, Y_sample

In [9]:
@jit
def gradlogistic_numba(X,Y,theta,n,V):  
    #theta is updated
    #X,Y are samples from the minibatch
    #n is total number of observations
    #V should be provided ahead, specified here
    Vinv = np.linalg.inv(V)
    d1 = -np.diag(Y-1/(1+np.exp(-X@theta)))@X
    d2 = Vinv@theta
    d1_avg = d1.mean(axis = 0)
    return d1_avg*n + d2

In [10]:
@jit([float64[:,:](float64[:],float64[:,:],float64[:],int64,float64[:],float64[:,:],float64[:,:],float64[:,:],float64,int64,int64)],cache = True)
def sghmc_numba(theta0,X,Y,nbatch,gradU,M,C,V,eps,step = 10, niter = 10):
    B = 1/2 * V * eps
    sigma = np.sqrt(2*eps*(C-B))
    n, p = X.shape
    theta = theta0 #set an initial value of theta
    thetas =np.zeros([step,p])
    Minv = np.linalg.inv(M)
    np.random.seed(10)
    #simulate dynamics
    for t in range(step):
        r = np.random.multivariate_normal(np.zeros(p),np.sqrt(M))
        for i in range(niter):
            theta = theta + eps*Minv@r
            X_sample,Y_sample = batch_numba(X,Y,nbatch)
            r =  r - eps*gradU(X_sample, Y_sample,theta,n,V) - eps*C @ Minv @ r 
            + np.random.multivariate_normal(np.zeros(p),sigma,1).ravel()
        thetas[t,:] = theta
    return thetas

## Comparison

In [11]:
theta0 = np.zeros(5)
M = C = np.eye(5)
nbatch=500
eps=.001
V = np.diag([20,20,20,20,20])

In [12]:
def gradlogistic(X,Y,theta,n,V):  
    #theta is updated
    #X,Y are samples from the minibatch
    #n is total number of observations
    #V should be provided ahead, specified here
    Vinv = np.linalg.inv(V)
    d1 = -np.diag(Y-1/(1+np.exp(-X@theta)))@X
    d2 = Vinv@theta
    d1_avg = d1.mean(axis = 0)
    return d1_avg*n + d2

In [13]:
@jit([float64[:,:](float64[:],float64[:,:],float64[:],int64,float64[:],float64[:,:],float64[:,:],float64[:,:],float64,int64,int64)],cache = True)
def sghmc_batch_numba(theta0,X,Y,nbatch,gradU,M,C,V,eps,step = 10, niter = 10):
    B = 1/2 * V * eps
    sigma = np.sqrt(2*eps*(C-B))
    n, p = X.shape
    theta = theta0 #set an initial value of theta
    thetas =np.zeros([step,p])
    Minv = np.linalg.inv(M)
    np.random.seed(10)
    #simulate dynamics
    for t in range(step):
        r = np.random.multivariate_normal(np.zeros(p),np.sqrt(M))
        for i in range(niter):
            theta = theta + eps*Minv@r
            idx = np.random.choice(n, nbatch, replace = False)
            X_sample = X[idx,:]
            Y_sample = Y[idx]
            r =  r - eps*gradU(X_sample, Y_sample,theta,n,V) - eps*C @ Minv @ r 
            + np.random.multivariate_normal(np.zeros(p),sigma,1).ravel()
        thetas[t,:] = theta
    return thetas

In [14]:
%timeit sghmc.sghmc(np.zeros(5),X_train,Y_train,nbatch,gradlogistic,M,C,V,eps,200,30)

7.38 s ± 655 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%timeit sghmc_batch_numba(np.zeros(5),X_train,Y_train,nbatch,gradlogistic_numba,M,C,V,eps,200,30)

8.99 s ± 614 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit sghmc_numba(np.zeros(5),X_train,Y_train,nbatch,gradlogistic_numba,M,C,V,eps,200,30)

4.94 s ± 140 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%prun -q -D optimized.prof  sghmc_numba(np.zeros(5),X_train,Y_train,nbatch,gradlogistic_numba,M,C,V,eps,200,30)

 
*** Profile stats marshalled to file 'optimized.prof'. 


In [18]:
import pstats
p = pstats.Stats('optimized.prof')
p.print_stats()
pass

Tue Apr 28 17:52:19 2020    optimized.prof

         916930 function calls (885889 primitive calls) in 5.980 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12400    0.003    0.000    0.003    0.000 {method 'append' of 'list' objects}
    31012    0.010    0.000    0.010    0.000 {method 'get' of 'dict' objects}
     6200    0.002    0.000    0.002    0.000 {method 'pop' of 'dict' objects}
    24800    0.006    0.000    0.006    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 {method 'values' of 'collections.OrderedDict' objects}
       11    0.000    0.000    0.000    0.000 {built-in method __new__ of type object at 0x10d7e9568}
        8    0.000    0.000    0.000    0.000 {method 'rpartition' of 'str' objects}
    18400    0.018    0.000    0.018    0.000 {built-in method builtins.abs}
        3    0.000    0.000    0.000    0.000 {built-in method builtins.bin}
        1    